In [1]:
import tensorflow as tf
import numpy as np

import tf_encrypted as tfe

In [2]:
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.models import Sequential

## Plaintext model

In [3]:
model_k = Sequential([Dense(5, batch_input_shape=[1,32]), 
                      Dense(2)])

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
data = np.ones(shape=[1, 32]).astype('float')
model_k.predict(data)

array([[0.03096867, 1.7848865 ]], dtype=float32)

## Create tfe keras model based on keras config and set weights

In [5]:
keras_weights = model_k.get_weights()
config = model_k.get_config()

In [6]:
tf.reset_default_graph()

with tfe.protocol.SecureNN():
    tfe_model = tfe.keras.Sequential([])
    tfe_model.from_config(config)
    x = tfe.define_private_variable(data)
    
    y = tfe_model(x)

In [7]:
tfe_model.layers[0].weights

[PondPrivateVariable(shape=(32, 5)), PondPrivateVariable(shape=(5,))]

In [8]:
with tfe.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tfe_model.set_weights(keras_weights, sess)
    actual = sess.run(y.reveal())

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2']


[<tf_encrypted.keras.layers.dense.Dense object at 0x135e6c208>, <tf_encrypted.keras.layers.dense.Dense object at 0x135e6c240>]


In [9]:
print(actual)

[[ 0.77838744 -1.13519281]]


In [10]:
keras_weights = model_k.get_weights()
config = model_k.get_config()

In [11]:
def update_weights(tfe_model, sess):
    
#     w1 = tfe_model.layers[0].kernel
#     b1 = tfe_model.layers[0].bias
#     w2 = tfe_model.layers[1].kernel
#     b2 = tfe_model.layers[1].bias
    
#     w1 = tfe_model.weights[0]
#     b1 = tfe_model.weights[1]
#     w2 = tfe_model.weights[2]
#     b2 = tfe_model.weights[3]
    
    w1 = tfe_model.layers[0].weights[0]
    b1 = tfe_model.layers[0].weights[1]
    w2 = tfe_model.layers[1].weights[0]
    b2 = tfe_model.layers[1].weights[1]
    
    w1_up = tfe.define_private_variable(keras_weights[0])
    b1_up = tfe.define_private_variable(keras_weights[1])
    w2_up = tfe.define_private_variable(keras_weights[2])
    b2_up = tfe.define_private_variable(keras_weights[3])
    
    sess.run(tf.global_variables_initializer())
    sess.run(tfe.assign(w1, w1_up))
    sess.run(tfe.assign(b1, b1_up))
    sess.run(tfe.assign(w2, w2_up))
    sess.run(tfe.assign(b2, b2_up))
    

In [12]:
tf.reset_default_graph()

with tfe.protocol.SecureNN():
    tfe_model = tfe.keras.Sequential([])
    tfe_model.from_config(config)
    x = tfe.define_private_variable(data)
    
    y = tfe_model(x)

with tfe.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    update_weights(tfe_model, sess)
    
    actual = sess.run(y.reveal())
    
print(actual)

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2']


[[0.03109282 1.78524615]]
